# Nerves Segmentation

## Importing libraries

Importing libraries to access directory, to load image, to use array, to plot, to process image, and generate random variables

In [1]:
import os, cv2, csv, random, time
from pathlib import Path
from os import listdir
from os.path import isfile, join
import openslide
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import binary_closing, binary_opening, binary_dilation, binary_erosion
from skimage.color import rgb2hsv
from skimage.transform import resize
from PIL import Image, ImageDraw
import scipy.misc    
from itertools import combinations, permutations
 
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda,Reshape
from keras.layers import Input,LeakyReLU, Dense, Layer,LocallyConnected1D,Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D,AveragePooling2D
from keras.layers import Conv1D,Conv2D, Conv3D, Conv2DTranspose,ConvLSTM2D, SimpleRNN, LSTM, Permute
from keras.layers import UpSampling2D, merge,Reshape
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import regularizers
from keras.engine import InputLayer
from keras.callbacks import ModelCheckpoint
 

C:\Users\Indri92\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Importing Functions

In [2]:
from utils import getRowsFromCSV,extractBBWithSizeTh,paddedMorphologicalOperation,boxOverlapCheck
from utils import filterColourRange,drawPredBoxOnImage
from utils import scoringRoiWrtAnnot,combineBoxes,processCC
from utils import saveImageToJpg, savingCSVFile,boundingBoxOnWSI,saveImageFromRoiPatchesWBoxToFolder

from preprocessing import preprocessListWSI
from model import get_CNN
from predict import printscoresDict,filterListWSI
from directDL import predictDirectDL, directPredictListWSI, directDLResults

## Listing Training, Validation and Testing Files

In [3]:
demoFiles=['10039','10097','10114']
print("Number of demo files is "+str(len(demoFiles)))

testingFiles=['10012', '10023', '10029', '10036','10039', '10049', '10064', '10067',
       '10071', '10072', '10073', '10078', '10087', '10088', '10093',
       '10096', '10097', '10102', '10113', '10114', '10116', '10121']
print("Number of testing files is "+str(len(testingFiles)))


Number of demo files is 3
Number of testing files is 22


## Defining Path for Source Files

In [4]:
fileDir=os.path.join(os.getcwd(),"dataFiles")
slicePath=os.path.join(fileDir,"svs files")

#annotFile.csv consists of fileName and bounding box coordinate [x, y, w, h] of the annotations
annotPath=os.path.join(fileDir,"annot csv") 

#sArea.csv consists of fileName and bounding box coordinate [x, y, w, h] of the slide area to be processed
sArea=getRowsFromCSV(os.path.join(fileDir,"sArea.csv"),'Dict')

## Defining Mean and Std

In [5]:
pxlToUm=0.5036
xTrainMean=np.array([2.13762123e+02, 2.07595257e+02, 2.03273641e+02])
xTrainStd=np.array([36.67175097, 39.33090933, 42.27675675])

## Loading CNN Model (Trained with HandPicked data and shifted augmentation)

In [6]:
#input pretrained model path
pretrainedModelPath=os.path.join(fileDir,"pretrained models","pretrained_CNN.h5")
#Load CNN model
model=get_CNN((160,160), chN=3, lr = 1e-4, loss='binary_crossentropy', metrics=['accuracy'], modelFile=pretrainedModelPath)

#Displaying model architecture summary
model.summary()

Model loaded: D:\UniData\Projects\NervesSegmentationProject\dataFiles\pretrained models\pretrained_CNN.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 160, 160, 3)  12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 160, 160, 16) 448         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 160, 160, 16) 64          conv2d_1[0][0]                   
___

D:\UniData\Projects\NervesSegmentationProject\model.py:152: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  model = Model(input = inputs, output = [finalLayer,fc1c])


## Direct DL Implementation

In [ ]:
#Assign file list with a list of file to be processed
fileList=testingFiles.copy()

#Get predictions
fileROIs,fileScores=directPredictListWSI(model, xTrainMean, xTrainStd, slicePath,fileList, annotPath=annotPath, sArea=sArea)
#Getting the scores and saves the predictions
testingScores=directDLResults(slicePath, fileList, fileROIs, fileScores, annotPath=annotPath, fileDir=fileDir)
#Print testing scores
printscoresDict(testingScores)

## Demo

In [ ]:
from demo import demoFile

fileList=demoFiles.copy()

#Get the area of the slide to be processed for demo as size of the whole slide is too big.
dArea=getRowsFromCSV(os.path.join(fileDir,"sArea_Demo.csv"),'Dict')

### Draw and visualise prediction on demo file 1 (Scroll for complete results)

In [ ]:
print("Processing file: "+str(fileList[0]))
demoFile(fileList[0], dArea, fileDir, slicePath, annotPath)

### Draw and visualise prediction on demo file 2 (Scroll for complete results)

In [ ]:
print("Processing file: "+str(fileList[1]))
demoFile(fileList[1], dArea, fileDir, slicePath, annotPath)

### Draw and visualise prediction on demo file 3 (Scroll for complete results)

In [ ]:
print("Processing file: "+str(fileList[2]))
demoFile(fileList[2], dArea, fileDir, slicePath, annotPath)